# 🪄 Phase 5: GenAI Personalization Engine
**Project:** "Olist-Next" Retention Engine
**Goal:** Generate hyper-personalized retention emails using Llama 3.

## 🧠 Strategy (The "Brain")
We don't just send generic spam. We use **Context-Aware AI**:
1.  **Targeting:** High-Value Customers (`>$100`) who are At-Risk.
2.  **Sentiment Adaptation:**
    * **Low Score (< 3):** The AI will write an *apologetic* email asking for a second chance.
    * **High Score (> 4):** The AI will write a *grateful* email asking for a referral.
    * **Neutral:** The AI will focus on a "We Miss You" discount.
3.  **Model:** `databricks-meta-llama-3-1-70b-instruct` via MLflow Deployments.

### Target Audience Filtering 
Select the top 10 high-value, at-risk customers.

In [0]:
from pyspark.sql.functions import col, desc

# Set Context
spark.sql("USE CATALOG olist_hackathon")

# 1. Load Gold Data
df_gold = spark.table("gold.customer_360")

# 2. Filter Logic
# - At Risk (is_churn_risk = 1)
# - High Value (monetary_value > 100)
# - Limit to 10 for Demo speed
df_targets = (
    df_gold
    .filter((col("is_churn_risk") == 1) & (col("monetary_value") > 100))
    .orderBy(desc("monetary_value"))
    .limit(10)
    .select(
        "customer_unique_id", 
        "favorite_category", 
        "monetary_value", 
        "avg_review_score"
    )
)

print("🎯 Target Audience Selected (Top 10 High-Value/At-Risk):")
display(df_targets)

### Advanced Prompt Engineering 
Here we define the logic to change the prompt based on customer satisfaction (Review Score).

In [0]:
from pyspark.sql.functions import udf, concat, lit, when
from pyspark.sql.types import StringType

# Define Prompt Logic as a UDF (User Defined Function)
# This creates the specific instruction for Llama 3 based on the user's history
def build_prompt_udf(category, spend, score):
    
    # 1. Determine Tone based on Review Score
    if score <= 3:
        tone = "apologetic, humble, and concerned. We evidently failed them previously."
        angle = "Ask for a second chance to prove we have improved."
    elif score >= 4.5:
        tone = "celebratory, warm, and grateful."
        angle = "Thank them for being a star customer. Ask if they would refer a friend."
    else:
        tone = "friendly and inviting."
        angle = "Tell them we miss them."

    # 2. Construct the Prompt
    prompt = f"""
    You are a Senior Customer Success Manager at Olist. Write a personalized retention email to a customer.
    
    Customer Profile:
    - Favorite Category: {category}
    - Total Spend: ${spend:.2f}
    - Average Review Score: {score}/5.0
    
    Instructions:
    - Tone: {tone}
    - Strategy: {angle}
    - Offer: Include a unique code 'OLIST_VIP' for 15% off.
    - Constraint: Keep it under 100 words. Do not include a subject line.
    """
    return prompt

# Register UDF
spark.udf.register("build_prompt", build_prompt_udf, StringType())

# Apply the Prompt Logic
df_with_prompts = df_targets.withColumn(
    "llm_prompt", 
    udf(build_prompt_udf, StringType())(col("favorite_category"), col("monetary_value"), col("avg_review_score"))
)

# Verify the Personalization
print("📝 Reviewing Generated Prompts (Notice the different Tones):")
display(df_with_prompts.select("avg_review_score", "llm_prompt"))

### The Generative Loop (Llama 3 Inference) 
We use pandas_udf to call the Databricks Foundation Model API efficiently.

In [0]:
import mlflow.deployments
import pandas as pd
from pyspark.sql.functions import pandas_udf, col
from typing import Iterator

# Use the Llama 3.3 model you found
ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"

@pandas_udf("string")
def generate_email_udf(prompts: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """
    Calls the Databricks Foundation Model Endpoint (Sequential/Robust Mode).
    Fixes 400 Error by sending strict OpenAI-format requests.
    """
    # Initialize Client
    client = mlflow.deployments.get_deploy_client("databricks")
    
    for batch in prompts:
        batch_output = []
        
        # Loop through each prompt in the batch individually
        for prompt in batch:
            # 1. Construct strict OpenAI-compatible payload
            # The endpoint expects 'messages' at the root
            payload = {
                "messages": [
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": 150,  # Optional: Limit length
                "temperature": 0.7  # Optional: Creativity
            }
            
            # 2. Call API
            try:
                # We pass the dictionary directly as 'inputs'
                response = client.predict(endpoint=ENDPOINT_NAME, inputs=payload)
                
                # Extract text (Standard OpenAI format response)
                text = response['choices'][0]['message']['content']
                batch_output.append(text)
                
            except Exception as e:
                # Error Handling
                batch_output.append(f"Error: {str(e)}")
        
        yield pd.Series(batch_output)

print(f"⏳ Calling GenAI Endpoint: {ENDPOINT_NAME}...")

# Run Inference
df_final_campaign = df_with_prompts.withColumn("generated_email", generate_email_udf(col("llm_prompt")))

# Show the Magic
display(df_final_campaign.select("customer_unique_id", "avg_review_score", "generated_email"))

In [0]:
import mlflow.deployments

client = mlflow.deployments.get_deploy_client("databricks")

print("🔍 Scanning for available Foundation Models...")
try:
    endpoints = client.list_endpoints()
    found = False
    for e in endpoints:
        # Show all available endpoint IDs and names
        print(f"ID: {e.get('id', 'N/A')} | Name: {e.get('name', 'N/A')}")
        if e.get('name', '').startswith("databricks-") and ("instruct" in e.get('name', '') or "chat" in e.get('name', '')):
            print(f"✅ AVAILABLE: {e.get('name')}")
            found = True
    
    if not found:
        print("❌ No 'databricks-' foundation models found. You may need to enable them in the 'Serving' tab.")
        
except Exception as e:
    print(f"Error listing endpoints: {e}")

### Save to Gold 
Finalize the campaign.

In [0]:
# 1. Save the Campaign to the Gold Layer
target_table = "olist_hackathon.gold.retention_campaigns"
print(f"⏳ Saving final campaign to {target_table}...")

(df_final_campaign
 .select("customer_unique_id", "favorite_category", "monetary_value", "avg_review_score", "generated_email")
 .write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(target_table)
)

print("✅ Campaign Data Saved!")

# 2. The "Victory Lap" - Display a Campaign Summary for Management
# We use Spark SQL to show the potential impact of this campaign
summary_query = """
    SELECT 
        CASE 
            WHEN avg_review_score >= 4.5 THEN 'Promoters (Referral Offer)'
            WHEN avg_review_score <= 3.0 THEN 'Detractors (Apology Offer)'
            ELSE 'Neutrals (Standard Offer)'
        END as Segment,
        count(*) as Customer_Count,
        sum(monetary_value) as Total_Revenue_At_Risk
    FROM olist_hackathon.gold.retention_campaigns
    GROUP BY 1
    ORDER BY Total_Revenue_At_Risk DESC
"""

print("\n📊 Campaign Dashboard:")
display(spark.sql(summary_query))